In [26]:
import os
import cv2

# create an empty dictionary to hold the image data
bills = {'MX020':[],'MX050':[],'MX100':[],'MX200':[],'MX500':[]}

target_size = (224, 224)
# load the image data into the dictionary
for filename in os.listdir('billetes'):
    for bill in bills:
        if bill in filename:
            img = cv2.imread(os.path.join('billetes',filename))
            img = cv2.resize(img, target_size)
            if img is not None:
                bills[bill].append(img)


In [27]:
from sklearn.model_selection import train_test_split
import numpy as np

# combine the image data into a single array for each class
X_020 = np.array(bills['MX020'])
X_050 = np.array(bills['MX050'])
X_100 = np.array(bills['MX100'])
X_200 = np.array(bills['MX200'])
X_500 = np.array(bills['MX500'])

# create corresponding target arrays
y_020 = np.zeros(X_020.shape[0], dtype=np.int)
y_050 = np.ones(X_050.shape[0], dtype=np.int)
y_100 = np.full(X_100.shape[0], 2, dtype=np.int)
y_200 = np.full(X_200.shape[0], 3, dtype=np.int)
y_500 = np.full(X_500.shape[0], 4, dtype=np.int)

# concatenate the data and targets into a single array
X = np.concatenate((X_020, X_050, X_100, X_200, X_500), axis=0)
y = np.concatenate((y_020, y_050, y_100, y_200, y_500), axis=0)

# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




C:\Users\angel\AppData\Local\Temp\ipykernel_18904\4164461632.py:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_020 = np.zeros(X_020.shape[0], dtype=np.int)
C:\Users\angel\AppData\Local\Temp\ipykernel_18904\4164461632.py:13: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the

In [30]:
from keras_applications import resnet50
from keras_applications.resnet50 import preprocess_input


# create a ResNet50 model to extract features from the images
#base_model = resnet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#base_model = resnet50.ResNet50(weights='imagenet', include_top=False)
base_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# extract features from the training set

X_train_features = base_model.predict(preprocess_input(X_train))
X_train_features = X_train_features.reshape(X_train_features.shape[0], -1)

# extract features from the test set
X_test_features = base_model.predict(preprocess_input(X_test))
X_test_features = X_test_features.reshape(X_test_features.shape[0], -1)


AttributeError: 'NoneType' object has no attribute 'image_data_format'

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# create a RandomForestClassifier and fit it to the training data
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_features, y_train)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# make predictions on the test set and compute evaluation metrics
y_pred = clf.predict(X_test_features)
acc = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
# print the evaluation metrics
print("Accuracy: {:.4f}".format(acc))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 score: {:.4f}".format(f1))


In [ ]:
import cv2
import matplotlib.pyplot as plt

# create a dictionary to map target values to bill denominations
bill_names = {0: 'MX020', 1: 'MX050', 2: 'MX100', 3: 'MX200', 4: 'MX500'}
cap = cv2.VideoCapture(0)
# create a function to classify images from the webcam
while True:
    # capture an image from the webcam

    ret, frame = cap.read()


    # preprocess the image and extract features using the ResNet50 model
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)
    features = base_model.predict(x)
    features = features.reshape(features.shape[0], -1)

    # make a prediction using the trained classifier
    y_pred = clf.predict(features)[0]
    bill_name = bill_names[y_pred]

    # display the image and the predicted bill denomination
    cv2.putText(frame, bill_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('frame',frame)


    if(cv2.waitKey(1) & 0xFF == ord('q')):
        break

cap.release()
cv2.destroyAllWindows()
